In [1]:
import sys
sys.path.append("../../../")
from typing import List
from model.map_pose import MapPose
from model.waypoint import Waypoint
from model.world_pose import WorldPose
from planner.local_planner.local_planner import LocalPlannerType, PlanningResult
from data.coordinate_converter import CoordinateConverter
from utils.logging import Telemetry
from planner.goal_point_discover import GoalPointDiscover, GoalPointDiscoverResult
from vision.occupancy_grid_cuda import OccupancyGrid
from model.physical_parameters import PhysicalParameters
import sys, os
sys.path.append("../../")
from carlasim.carla_client import CarlaClient
#from slam.slam import SLAM
from model.map_pose import MapPose
from model.world_pose import WorldPose
from carlasim.debug.carla_debug import CarlaDebug
from carlasim.carla_ego_car import CarlaEgoCar
import time

COORD_ORIGIN = WorldPose(lat=-4.303359446566901e-09, 
                      lon=-1.5848012769283334e-08,
                      alt=1.0149892568588257,
                      heading=0)

PLAN_TIMEOUT = -1
PLANNER_TYPE = LocalPlannerType.Overtaker


def show_goal_point_discover_plan (seq: int) -> tuple[GoalPointDiscoverResult, PlanningResult]:
    coord = CoordinateConverter(COORD_ORIGIN)
    local_goal_discover = GoalPointDiscover(coord)

    result = Telemetry.read_planning_result(seq)
    if result is None:
        return False
    
    bev = Telemetry.read_planning_bev(seq)

    og = OccupancyGrid(
                frame=bev,
                minimal_distance_x=PhysicalParameters.MIN_DISTANCE_WIDTH_PX,
                minimal_distance_z=PhysicalParameters.MIN_DISTANCE_HEIGHT_PX,
                lower_bound=PhysicalParameters.EGO_LOWER_BOUND,
                upper_bound=PhysicalParameters.EGO_UPPER_BOUND
            )

    res = local_goal_discover.find_goal(
        og=og,
        current_pose=result.ego_location,
        goal_pose=result.map_goal,
        next_goal_pose=result.map_next_goal
    )
    
   
    if res is None:
        print(f"no goal was found for seq: {seq}")
        return
    
    projected_goal = coord.clip(coord.convert_map_to_waypoint(result.ego_location, result.map_goal))
    
    print(f"[{seq}] projected local goal: ({projected_goal.x}, {projected_goal.z})")
    if res.goal is None:
        print(f"[{seq}] chosen local goal: NONE")
        return
    
    print(f"[{seq}] chosen local goal: ({res.goal.x}, {res.goal.z})")
    
    h = Waypoint.compute_heading(res.start, res.goal)
    print(f"\tdirect heading:{h:.4} degrees")
    print(f"\tchosen heading: {res.goal.heading:} degrees")
    
    return res, result

client = CarlaClient(town='Town07')
debug = CarlaDebug(client)
ego = CarlaEgoCar(client)

In [2]:
goal_result, recorded_result = show_goal_point_discover_plan(15)


[15] projected local goal: (132, 53)
[15] chosen local goal: (125, 53)
	direct heading:-3.18 degrees
	chosen heading: -22.988716802080646 degrees


In [3]:
pose = recorded_result.ego_location.clone()
ego.set_pose(pose.x, pose.y, 2, pose.heading)


In [4]:
debug.show_map_pose(recorded_result.ego_location, mark='1', lifetime=60)
debug.show_map_pose(recorded_result.map_goal, mark='2', lifetime=60)
debug.show_map_pose(recorded_result.map_next_goal, mark='3', lifetime=60)

In [5]:
coord = CoordinateConverter(COORD_ORIGIN)
m = coord.convert_waypoint_to_map_pose(
    recorded_result.ego_location,
    goal_result.goal
)
debug.show_map_pose(m, mark='4', lifetime=30)

m.heading = pose.heading + goal_result.goal.heading
print(f"pose heading = {m.heading }")
debug.show_map_pose(m, True, lifetime=30)


pose heading = -23.15457982428886
